In [1]:
## Exploratory Data Analysis for the couple text-message insights

In [2]:
import pandas as pd
import nltk
import json

In [3]:
with open('result.json') as f:
    chatlog_raw = json.load(f)


In [4]:
df_chatlog = pd.DataFrame(chatlog_raw['messages'])

In [5]:
df_chatlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192772 entries, 0 to 192771
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            192772 non-null  int64  
 1   type                          192772 non-null  object 
 2   date                          192772 non-null  object 
 3   from                          192741 non-null  object 
 4   from_id                       192741 non-null  float64
 5   text                          192772 non-null  object 
 6   reply_to_message_id           10346 non-null   float64
 7   edited                        4058 non-null    object 
 8   forwarded_from                915 non-null     object 
 9   photo                         3065 non-null    object 
 10  width                         13183 non-null   float64
 11  height                        13183 non-null   float64
 12  file                          10562 non-null

In [6]:
df_chatlog['type'].unique()

array(['message', 'service'], dtype=object)

In [7]:
# records with type == 'service' are records of when calls were made
# -> filter records 
df_mo = df_chatlog[df_chatlog['type'].map(lambda x: x=='message')]

In [11]:
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer 

stop_words = set(stopwords.words('english')) 
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')

In [20]:
def process_message(txt):
    _processed = []
    
    if type(txt) == list:
        txt = [item['text'] if type(item) == dict else item for item in txt]
        txt = ' '.join(txt)
    
    _tokens = tokenizer.tokenize(txt)
    for token in _tokens:
        _lower = token.lower()
        if _lower in stop_words:
            continue
        else:
            _processed.append(_lower)
            
    return _processed

df_mo['text_processed'] = df_mo['text'].map(lambda x: process_message(x))

<ipython-input-20-cb3661f48e0f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mo['text_processed'] = df_mo['text'].map(lambda x: process_message(x))


In [21]:
df_mo['text_processed'].head()

0                         [hey, melissa, !, alfred, 🍉]
1                        ['s, friday, evening, far, ?]
2    ['m, laying, floor, living, room, "warm, cold,...
3                               [bass, feels, nice, 😍]
4                                      [hello, alfred]
Name: text_processed, dtype: object

In [25]:
fdist = nltk.FreqDist()
for index, item in df_mo['text_processed'].iteritems():
    for word in item:
        fdist[word] += 1
        

In [28]:
fdist.most_common(50)

[('u', 20093),
 ('bb', 10432),
 ('?', 10397),
 ('love', 5607),
 ('hehe', 5540),
 ('like', 5257),
 (',', 4894),
 ('yes', 4635),
 ('babe', 4585),
 ("'s", 4460),
 ('go', 4404),
 ('mi', 4304),
 ('think', 4016),
 ('!', 4008),
 ('n', 3916),
 ('ok', 3793),
 ('❤️', 3691),
 ('im', 3678),
 ("'t", 3477),
 ('ur', 3464),
 ('.', 3403),
 ('also', 3396),
 ('good', 3349),
 ("'m", 3304),
 ('time', 3286),
 ('hahaha', 3205),
 ('r', 3166),
 ('boo', 3108),
 ('get', 2870),
 ('okay', 2797),
 ('haha', 2529),
 ('...', 2415),
 ('!!!', 2411),
 ('work', 2293),
 ("'ll", 2217),
 ('today', 2139),
 ('nice', 2080),
 ('see', 2032),
 ('!!', 2004),
 ("'re", 1951),
 ('w', 1936),
 ('one', 1900),
 ('gonna', 1895),
 ('sleep', 1863),
 ('got', 1859),
 ('much', 1789),
 ('rly', 1766),
 ('omg', 1762),
 ('day', 1755),
 ('sure', 1668)]